<a id='0'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:250%; text-align:center; border-radius: 15px 50px;">Credit Card Dataset🖋📝 - EDA📚 & Machine Learning Model🎯 </p>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style

%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import squarify

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
# read data
data = pd.read_csv('../input/ccdata/CC GENERAL.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
#Count missing variable
data.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# split features into Categorical and Numerical 
catogrical = [x for x in data.columns if data[x].dtype == "object"]
numeric = [x for x in data.columns if data[x].dtype == "float64"]

In [ ]:
#filling null or missing values with more frequent values
for i in catogrical:
    data[i].fillna(data[i].mode()[0], inplace=True)

In [ ]:
# filling null or missing values with mean or average 

for i in numeric:
    data[i].fillna(data[i].mean(), inplace = True)

In [ ]:
#let's check null values again 
sns.heatmap(data.isnull())

In [ ]:
data.dtypes

In [ ]:
#Dropping CUST_ID feature
data.drop(['CUST_ID'], axis=1, inplace=True)

<a id='2'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">Data Vizualization🎨</p>

In [ ]:
# see how the data is distributed.
data.hist(figsize = (20,20))

In [ ]:
sns.kdeplot(data['MINIMUM_PAYMENTS'], shade=True) 
plt.title('Kernel Density Estimation Plot') 
data['MINIMUM_PAYMENTS']=data['MINIMUM_PAYMENTS'].fillna(data['MINIMUM_PAYMENTS'].median())

In [ ]:
data_dist=data.iloc[:,1:17]
data_columns=data_dist.columns

r,c=0,0
fig, axes=plt.subplots(4,4, figsize=(20,16))
#plt.tight_layout()
for i in data_columns:
    sns.distplot(data[i], ax=axes[r,c])
    c += 1
    if c == 4: 
        r += 1
        c=0
    if r == 4: break
plt.suptitle('Kernel Density Estimation Plot', fontsize=15)

In [ ]:
plt.figure(dpi = 100, figsize = (5,4))
print("Joint plot of Month of Absence with Other Variables ==> \n")
for i in  data.columns:
    if i != 'CASH_ADVANCE' and i != 'PURCHASES':
        print(f"Correlation between PURCHASES and {i} ==> ",data.corr().loc['PURCHASES'][i])
        sns.jointplot(x='PURCHASES',y=i,data=data,kind = 'reg',color = 'purple')
        plt.show()

In [ ]:
corr_coef=data[1:].corr()

# Heatmap
plt.figure(figsize=(25, 25))
sns.heatmap(corr_coef, cmap='Greens', annot=True, annot_kws={'size':14},
            xticklabels=corr_coef.columns,
            yticklabels=corr_coef.columns)
plt.title('Correlation Matrix')

# Find out feature pairs whose coefficient >= 0.7
corr_cols=corr_coef.columns.to_list() 
signif_corr=[]
for i in range(len(corr_cols)):
    col=corr_cols[i]
    signif_corr.append(abs(corr_coef[col])[abs(corr_coef[col]) >= 0.7])
signif_corr_df=pd.DataFrame(signif_corr)
#signif_corr_df['PURCHASES']['ONEOFF_PURCHASES'] 

In [ ]:
import pandas_profiling as pp

pp.ProfileReport(data)

In [ ]:
#check outliers
plt.figure(figsize=(10,10))
sns.boxplot(data=data)
plt.xticks(rotation=90)

In [ ]:
#Drop outliers according to z-score
from scipy import stats
z = np.abs(stats.zscore(data))
print(z)

threshold = 3
print(np.where(z > 3))

data_o = data[(z < 3).all(axis=1)]

In [ ]:
#Drop outliers according to z-score
from scipy import stats
z = np.abs(stats.zscore(data))
print(z)

threshold = 3
print(np.where(z > 3))

data_o = data[(z < 3).all(axis=1)]

In [ ]:
#Drop outliers according to z-score
from scipy import stats
z = np.abs(stats.zscore(data))
print(z)

threshold = 3
print(np.where(z > 3))

data_o = data[(z < 3).all(axis=1)]

In [ ]:
#Drop outliers according to z-score
from scipy import stats
z = np.abs(stats.zscore(data))
print(z)

threshold = 3
print(np.where(z > 3))

data_o = data[(z < 3).all(axis=1)]

In [ ]:
data.shape

In [ ]:
data_o.shape


In [ ]:
#Normalize
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, normalize

# Get column names first
names = data_o.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(data_o)
scaled_df = pd.DataFrame(scaled_df, columns=names)
  
# Normalizing the Data 
normalized_df = normalize(scaled_df) 
  
# Converting the numpy array into a pandas DataFrame 
normalized_df = pd.DataFrame(normalized_df,columns=names) 

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(data=normalized_df)
plt.xticks(rotation=90)

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = normalized_df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["PURCHASES"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [ ]:
data.columns

In [ ]:
#Feature Selection using LassoCV

from sklearn.linear_model import LassoCV

#Feature Selection
X = normalized_df.drop("BALANCE",1)   #Feature Matrix
y = normalized_df["BALANCE"]          #Target Variable

reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  
      str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
X = X.drop(["PURCHASES",'PRC_FULL_PAYMENT','CASH_ADVANCE_TRX','INSTALLMENTS_PURCHASES','TENURE','PURCHASES_FREQUENCY'],1)   #Feature Matrix

In [ ]:
#KMeans Clustering
#Defining WCSS Elbow point
from sklearn.cluster import KMeans

wcss=[]
for i in range (1,30):
    kmeans=KMeans(i)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
wcss

In [ ]:
#Elbow Plot
plt.plot(range(1,30),wcss)
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
#Another Technique to define n_cluster

# Import the KElbowVisualizer method 
from yellowbrick.cluster import KElbowVisualizer

# Instantiate a scikit-learn K-Means model
model = KMeans(random_state=0)

# Instantiate the KElbowVisualizer with the number of clusters and the metric 
visualizer = KElbowVisualizer(model, k=(2,30), metric='silhouette', timings=False)

# Fit the data and visualize
visualizer.fit(X)    
visualizer.poof()  

In [ ]:
k_means_new=KMeans(5)
kmeans.fit(X)
cluster_new=X.copy()
cluster_new['cluster_pred']=k_means_new.fit_predict(X)
cluster_new.head()

In [ ]:
# Visualize cluster shapes in 3d.

cluster1=cluster_new.loc[cluster_new['cluster_pred'] == 0]
cluster2=cluster_new.loc[cluster_new['cluster_pred'] == 1]
cluster3=cluster_new.loc[cluster_new['cluster_pred'] == 2]
cluster4=cluster_new.loc[cluster_new['cluster_pred'] == 3]
cluster5=cluster_new.loc[cluster_new['cluster_pred'] == 4]


In [ ]:
facet = sns.lmplot(data=cluster_new, x='CREDIT_LIMIT', y='PAYMENTS',hue='cluster_pred', 
                   fit_reg=False, legend=True, legend_out=True)

## Hierarchical Clustering

In [ ]:
# importing all important libraries
import scipy.cluster.hierarchy as hcluster
from sklearn.cluster import AgglomerativeClustering


# Single linkage: 
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree


In [ ]:
features=normalized_df.columns

In [ ]:
# Average linkage

mergings = linkage(X, method="average", metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
#plotting dendogram
plt.figure(figsize=(50, 12))
dend=hcluster.dendrogram(hcluster.linkage(X,method='ward'))

In [ ]:
# Getting labels from Agglomearative Hierarchical clustering
hcluster = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')  
hcluster.fit_predict(X)
hcluster_label = hcluster.labels_

In [ ]:
hcluster_df = pd.DataFrame(X)
#adding hcluster labels in hcluster_df
hcluster_df['hcluster'] = hcluster_label
#first few rows of hcluster_df
hcluster_df.head()

In [ ]:
facet = sns.lmplot(data=hcluster_df, x='CASH_ADVANCE', y='PURCHASES_INSTALLMENTS_FREQUENCY',hue='hcluster', 
                   fit_reg=False, legend=True, legend_out=True)

In [ ]:
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8,9,10,11]

for num_clusters in range_n_clusters:
    # intialise hclustering
    # Getting labels from Agglomearative Hierarchical clustering
    hcluster = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='ward')  
    hcluster.fit_predict(X)
    hcluster_label = hcluster.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(X, hcluster_label)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))